# Example usage

To use `instrumentum` in a project:

In [1]:
import instrumentum

print(instrumentum.__version__)

0.1.2


In [12]:
import instrumentum.time_series as ts
import pandas as pd

data_file = "./sample_data/time_series.csv"
data_df = pd.read_csv(data_file)

ts.ts_print(data_df, col_target="ACQUIRED", col_key="CUSTOMER_KEY")

Total unique keys that with a possitive value:  2
Total records of ckeys that with a possitive value:  13
Total unique keys with no possitive value:  2
Total records of customers with no possitive values:  10
